In [2]:
import fluids
import thermo
import pandas as pd
import numpy as np 
import ht

**<u>Compressor Sizing</u>** <br>
For this example, we’ll perform the compressor calculations on the
following fluid:

Methane 0.5
Ethane 0.25
Propane 0.125
i-Butane 6.25e-002
n-Butane 6.25e-002

Note: Above Values are Mole fractions <br>
T inlet = 25 <sup>o</sup>C

P inlet = 200 kPa

Mass flow = 40,000 kg/hr

**Isentropic work Calculations**

$$W\  = \ \frac{k}{k - 1}ZRT_{1}\left\lbrack \left( \frac{P_{2}}{P_{1}} \right)^{(k - 1)/k} - 1 \right\rbrack/\eta_{isentropic}$$

**Isentropic Temperature rise**

$$\mathbf{T}_{\mathbf{2}}\mathbf{=}\mathbf{T}_{\mathbf{1}}\mathbf{+}\frac{\mathbf{\mathrm{\Delta}}\mathbf{T}_{\mathbf{s}}}{\eta_{isentropic}}\mathbf{=}\mathbf{T}_{\mathbf{1}}\mathbf{+}\frac{\mathbf{T}_{\mathbf{1}}}{\eta_{isentropic}}\mathbf{\lbrack}\left( \frac{P_{2}}{P_{1}} \right)^{(k - 1)/k} - 1\mathbf{\rbrack}$$

For ideal Case

$$\frac{\mathbf{T}_{\mathbf{2}}}{\mathbf{T}_{\mathbf{1}}}\  = \left( \frac{P_{2}}{P_{1}} \right)^{(k - 1)/k}\ $$

Where $\eta_{isentropic}$ = 1 and T<sub>2</sub> = T<sub>adiabatic</sub> 

**Relationship between polytropic and isentropic efficiency**

$$\eta_{isentropic} = \frac{\left( \frac{P_{2}}{P_{1}} \right)^{(k - 1)\text{/}k} - 1}{\left( \frac{P_{2}}{P_{1}} \right)^{(k - 1)\text{/}k*\eta_{polytropic}} - 1}$$

$$\eta_{polytropic} = \frac{(k - 1)*\ln\left( \frac{P_{2}}{P_{1}} \right)}{k*\ln{(\frac{1}{\eta_{isentropic}}(\eta_{isentropic} + \left( \frac{P_{2}}{P_{1}} \right)^{(k - 1)\text{/}k} - 1)}}$$

**Polytropic Exponent and Polytropic Efficiency**

$$\mathbf{n\  = \ }\frac{\mathbf{k*}\eta_{polytropic}}{\mathbf{1 - k*(1 -}\eta_{polytropic})}$$

$$\eta_{polytropic}\  = \ \frac{n*(k - 1)}{k*(n - 1)}$$

**Units** <br>
Temperatures in Kelvin, Pressures in kPa, work in j/mol

**Link: [More on Compressor Calculations and developped webapp](https://www.linkedin.com/posts/ahmed-hassan-aprco_tool-documentation-activity-7039387620459655168-m1-8/?utm_source=share&utm_medium=member_desktop)**

In [3]:
m_wt = 28.32
mass_flow = 40000 # kg/hr
mole_flow = mass_flow/m_wt 
T1 = 25+273.15
P1 = 200E3 #kPa
P2 = 800E3 #kPa
k = 1.16337674864382 # Isentropic exponent
power_consumed = fluids.isentropic_work_compression(P1=P1, P2=P2, T1=T1, k=k, eta=0.8) *2.77777778E-7*mole_flow*1000 # Convert j/mol to Kw/mol * mole flow kmol*1000
power_consumed_jPerMol = power_consumed/ (2.77777778E-7*mole_flow*1000)
efficiency = fluids.isentropic_work_compression(T1=T1, P1=P1, P2=P2, k=k, W=power_consumed_jPerMol) # Calculate efficiency
P2 = fluids.isentropic_work_compression(T1=T1, P1=P1, k=k, W=power_consumed_jPerMol, eta=0.8) # Calculate P2
P1 = fluids.isentropic_work_compression(T1=T1, P2=P2, k=k, W=power_consumed_jPerMol, eta=0.8) # Calculate P1
T2 = fluids.isentropic_T_rise_compression(T1=T1, P1=P1, P2=P2, k=k, eta=0.8) - 273.15 
T2_adiabatic = fluids.isentropic_T_rise_compression(T1=T1, P1=P1, P2=P2, k=k, eta=1) - 273.15
eta_p = fluids.isentropic_efficiency(P1=P1, P2=P2, k=k, eta_s=0.8) # with eta_s specified, returns polytropic efficiency

n = fluids.polytropic_exponent(k=k, eta_p=eta_p)

print('Calculated Consumed Power is {:.2f}'.format(power_consumed))
print('Calculated Isentropic Efficiency is {:.2f}'.format(efficiency*100))
print('Calculated P2 is {:.2f}'.format(P2))
print('Calculated P1 is {:.2f}'.format(P1))
print('Calculated T2 is {:.2f}'.format(T2))
print('Calculated Adiabatic T2 is {:.2f}'.format(T2_adiabatic))
print('Calculated Polytropic Efficiency is {:.2f}'.format(eta_p*100))
print('Calculated Polytropic exponent is {:.2f}'.format(n))

Calculated Consumed Power is 1860.63
Calculated Isentropic Efficiency is 80.00
Calculated P2 is 800000.00
Calculated P1 is 200000.00
Calculated T2 is 105.10
Calculated Adiabatic T2 is 89.08
Calculated Polytropic Efficiency is 81.81
Calculated Polytropic exponent is 1.21


**Connecting to HYSYS to compare results**

Code for connecting to hysys is a code snippet from Edgar Ivan's Code
 

**Check it out Is pretty awesome!: [Aspen_HYSYS_Python-Github](https://github.com/edgarsmdn/Aspen_HYSYS_Python/tree/main)** <br>
**Youtube Video: [ Connecting #aspenhysys with #python | Aspen HYSYS | Python | Connectivity ](https://www.youtube.com/watch?v=aXujkOoNrSg&ab_channel=QaziNasir)**

In [4]:
import os                          # Import operating system interface
import win32com.client as win32  # Import COM
 
def retrieve_equipment_spreadsheet(File_name,Spreadsheet_name,Unit_name):
    hyFilePath = os.path.abspath(File_name)
        
    # 2.0 Initialize  Aspen HYSYS application
    print(' ')
    print('Connecting to the Aspen HYSYS.... Please wait ')
    # This gets the registered name of Aspen HYSYS
    HyApp = win32.Dispatch('HYSYS.Application')

    # 3.0 Open Aspen HYSYS file

    HyCase = HyApp.SimulationCases.Open(hyFilePath)


    # else:
    # raise Exception('Argument for input variable "active" is not valid')
        
    # 4.0 Aspen HYSYS Environment Visible
    HyCase.Visible = 1

    # 5.0 Aspen HYSYS File Name
    HySysFile = HyCase.Title.Value
    print(' ')
    print('Aspen HYSYS file name:      ', HySysFile)

        
    # 7.0  Spreadsheets
    ss_dict = {}
    # for ss in Spreadsheets:
    spsh = HyCase.Flowsheet.Operations.Item(Spreadsheet_name)
    ss_dict[Spreadsheet_name] = spsh


    # 10.0 Unit operations
    uo_dict = {}
    # for uo in Units:
    unop = HyCase.Flowsheet.Operations.Item(Unit_name)
    uo_dict[Unit_name] = unop

    # 11.0 Collect everything into a class
    class Hysys:
        pass
    Hysys.SS                = ss_dict
    Hysys.UO                = uo_dict
    return Hysys.SS[Spreadsheet_name]
File_name         = 'water&gas.hsc'
Spreadsheet_name = 'CompressorSS'
Unit        = 'K-100'
spread_sheet = retrieve_equipment_spreadsheet(File_name,Spreadsheet_name,Unit)

print('Calculated Consumed Power is {:.2f}'.format(spread_sheet.Cell(1,4).CellValue))
print('Input Isentropic Efficiency is {:.2f}'.format(spread_sheet.Cell(1,0).CellValue))
print('Input P2 is {:.2f}'.format(spread_sheet.Cell(1,7).CellValue))
print('Input P1 is {:.2f}'.format(spread_sheet.Cell(1,6).CellValue))
print('Calculated T2 is {:.2f}'.format(spread_sheet.Cell(1,8).CellValue))
print('Calculated Polytropic Efficiency is {:.2f}'.format(spread_sheet.Cell(1,2).CellValue))
print('Calculated Polytropic exponent is {:.2f}'.format(spread_sheet.Cell(1,3).CellValue))
print('Calculated Isentropic exponent is {:.2f}'.format(spread_sheet.Cell(1,1).CellValue))

 
Connecting to the Aspen HYSYS.... Please wait 
 
Aspen HYSYS file name:       water&gas.hsc
Calculated Consumed Power is 1840.47
Input Isentropic Efficiency is 80.00
Input P2 is 800.00
Input P1 is 25.00
Calculated T2 is 109.77
Calculated Polytropic Efficiency is 81.83
Calculated Polytropic exponent is 1.21
Calculated Isentropic exponent is 1.16


**<u>Pump Sizing</u>**

**Fluids Docs for Pumps Calculations Link :**

$$Head = \ \frac{\mathrm{\Delta}P}{\rho.g}$$

$$P_{hydraulic\ power} = \frac{\rho.H.Q}{367000}$$

$$P_{Consumed\ power} = \frac{P_{hydraulic\ power}}{ƞ_{pump}}$$

$$ƞ_{pump}(Estimation) = \  - 0.316 + 0.24015*\ln(Q/3600) - 0.01199*\ln\left( ({Q/3600)}^{2} \right)$$

$$Pump\ Specific\ Speed\ (SS) = \ \frac{n\sqrt{Q/3600}}{H^{0.75}}$$

$$Pump\ Suction\ Specific\ Speed\ (SSS) = \ \frac{n\sqrt{Q/3600}}{{NPSHr}^{0.75}}$$

Units:

Q in m<sup>3</sup>/hr

H in m

ρ in kg/m<sup>3</sup>

P in kPa

Power in kW

n: pump speed in rpm

**Link: [More on Pumps' and Pumping Systems' Calculations and developped webapp](https://www.linkedin.com/posts/ahmed-hassan-aprco_pump-hydraulic-system-analysis-and-specs-activity-7137573783984386049-g1Xb/?utm_source=share&utm_medium=member_desktop)**

In [5]:
P_suction = 120E3  #kPa
P_discharge = 800E3 #kPa
Q = 60 #m3/hr 
Density =996.89 # kg/m3
# Viscosity = 0.890438973953439 #cP
dP = P_discharge - P_suction 
head = fluids.head_from_P(dP, Density)
estimated_efficiency = fluids.pump.Corripio_pump_efficiency(Q/3600)
Hydraulic_power = Q*head*Density/367000
Consumed_Power = Hydraulic_power/estimated_efficiency
power_margin = 1.15
Motor_Size = fluids.pump.motor_round_size(Consumed_Power*power_margin*1000)/1000
estimated_fullload_efficiency = fluids.pump.Corripio_motor_efficiency(Motor_Size*1000)

print('Calculated delta P {:.2f} kPa \nCalculated Head {:.2f} m'.format(dP/1000,head))
print('Estimated Efficiency is {:.2f} % \nCalculated Hyraulic Power {:.2f} kW \nCalculated Total Power {:.2f} kW'.format(estimated_efficiency*100,Hydraulic_power,Consumed_Power))
print('Estimated Motor Size is {:.2f} kW'.format(Motor_Size))
print('Estimated full load motor efficiency {:.2f} % '.format(estimated_fullload_efficiency*100))

Calculated delta P 680.00 kPa 
Calculated Head 69.56 m
Estimated Efficiency is 65.03 % 
Calculated Hyraulic Power 11.34 kW 
Calculated Total Power 17.43 kW
Estimated Motor Size is 22.37 kW
Estimated full load motor efficiency 88.74 % 


In [6]:
estimated_SS = fluids.pump.specific_speed(Q/3600, head, n=3600)
NPSHr = 4.0 # Net Positive Suction Head Required
estimated_SSS = fluids.pump.specific_speed(Q/3600, NPSHr, n=3600) # n = rpm
print('Calculated Pump Specific Speed is {:.2f} rpm*m^0.75/s^0.5'.format(estimated_SS))
print('Calculated Pump Suction Specific Speed is {:.2f} rpm*m^0.75/s^0.5'.format(estimated_SSS))

Calculated Pump Specific Speed is 19.30 rpm*m^0.75/s^0.5
Calculated Pump Suction Specific Speed is 164.32 rpm*m^0.75/s^0.5


In [7]:
Power = 35 / 1000 # Power, [W]
fluids.pump.CSA_motor_efficiency(Power, closed=True, poles=2, high_efficiency=True)

0.77

**Control Valve Sizing**
1. **Liquids** <br>

**<u>Incompressible fluids</u>**

**<u>Turbulent flow</u>**

**<u>Non-choked flow (without fittings)</u> (ΔP \<
F<sub>L</sub><sup>2</sup>(P<sub>1</sub>-F<sub>F</sub>P<sub>v</sub>) )**

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{1}}}\sqrt{\frac{\frac{\mathbf{\rho}_{\mathbf{1}}}{\mathbf{\rho}_{\mathbf{o}}}}{\mathbf{\mathrm{\Delta}P}}}$$

**<u>Non-choked flow (with fittings)</u> (ΔP \<
(F<sub>LP</sub>/F<sub>p</sub>)
<sup>2</sup>(P<sub>1</sub>-F<sub>F</sub>P<sub>v</sub>) )**

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{1}}\mathbf{F}_{\mathbf{P}}}\sqrt{\frac{\frac{\mathbf{\rho}_{\mathbf{1}}}{\mathbf{\rho}_{\mathbf{o}}}}{\mathbf{\mathrm{\Delta}P}}}$$

**<u>Choked flow (without fittings)</u> (ΔP \>
F<sub>L</sub><sup>2</sup>(P<sub>1</sub>-F<sub>F</sub>P<sub>v</sub>) )**

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{1}}\mathbf{F}_{\mathbf{L}}}\sqrt{\frac{\frac{\mathbf{\rho}_{\mathbf{1}}}{\mathbf{\rho}_{\mathbf{o}}}}{\mathbf{P}_{\mathbf{1}}\mathbf{-}\mathbf{F}_{\mathbf{F}}\mathbf{P}_{\mathbf{v}}}}$$

**<u>Choked flow (with fittings)</u> (ΔP \>
(F<sub>LP</sub>/F<sub>p</sub>)
<sup>2</sup>(P<sub>1</sub>-F<sub>F</sub>P<sub>v</sub>) )**

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{1}}\mathbf{F}_{\mathbf{LP}}}\sqrt{\frac{\frac{\mathbf{\rho}_{\mathbf{1}}}{\mathbf{\rho}_{\mathbf{o}}}}{\mathbf{P}_{\mathbf{1}}\mathbf{-}\mathbf{F}_{\mathbf{F}}\mathbf{P}_{\mathbf{v}}}}$$

**<u>Non-Turbulent flow (without fittings)</u>**

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{1}}\mathbf{F}_{\mathbf{R}}}\sqrt{\frac{\frac{\mathbf{\rho}_{\mathbf{1}}}{\mathbf{\rho}_{\mathbf{o}}}}{\mathbf{\mathrm{\Delta}P}}}$$

W in kg/hr

Q in m<sup>3</sup>/hr for liquids or Nm<sup>3</sup>/hr for gases

P in bar

ρ in kg/m<sup>3</sup>

ρ<sub>0 :</sub> 1000 kg/m<sup>3</sup>

N<sub>1</sub>: 0.865



**Link: [More on the Constants mentioned and developped webapp](https://www.linkedin.com/posts/ahmed-hassan-aprco_calculationsguidlines-for-control-valve-activity-7153106921200234496-Kya8/?utm_source=share&utm_medium=member_desktop)**

In [8]:
viscosity = 3.1472E-4 # Viscosity of the fluid [Pa*s]
density = 965.4 # Density of the liquid at the inlet [kg/m^3]
P_saturation = 70.1E3 # Saturation pressure of the fluid at inlet temperature [Pa]
P_Critical = 22120E3 # Critical pressure of the fluid [Pa]
P_inlet = 680E3 # Inlet pressure of the fluid before valves and reducers [Pa]
P_outlet = 220E3 # Outlet pressure of the fluid after valves and reducers [Pa]
D1 = 0.15 # Diameter of the pipe before the valve [m]
D2 = 0.15 # Diameter of the pipe after the valve [m]
d = 0.15 # Diameter of the valve [m]
Q = 0.1 # Volumetric flow rate of the fluid [m^3/s]

Calculated_Cv_Liquid =fluids.size_control_valve_l(rho=density, Psat=P_saturation, Pc=P_Critical, mu=viscosity,
P1=P_inlet, P2=P_outlet, Q=Q, D1=D1, D2=D2, d=d,FL=0.9, Fd=0.46)

print('Calculated Cv is {:.2f}'.format(Calculated_Cv_Liquid))

Calculated Cv is 165.00


2. Gases <br>
**<u>Compressible Flow</u>**

**<u>Turbulent flow</u>**

**<u>Non-choked flow (without fittings)</u> (ΔP/P<sub>1</sub> \<
F<sub>ϒ</sub>.x<sub>T</sub> )**

$$Cv\  = \ \frac{\mathbf{W}}{\mathbf{N}_{\mathbf{6}}\mathbf{Y}\sqrt{\mathbf{x}\mathbf{P}_{\mathbf{1}}\mathbf{\rho}_{\mathbf{1}}}}$$

$$Cv\  = \ \frac{\mathbf{W}}{\mathbf{N}_{\mathbf{8}}\mathbf{P}_{\mathbf{1}}\mathbf{Y}}\sqrt{\frac{\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{xM}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{9}}\mathbf{P}_{\mathbf{1}}\mathbf{Y}}\sqrt{\frac{\mathbf{M}\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{x}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{7}}\mathbf{P}_{\mathbf{1}}\mathbf{Y}}\sqrt{\frac{\mathbf{Gg}\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{x}}}$$

**<u>Non-choked flow (with fittings)</u> (ΔP/P<sub>1</sub> \<
F<sub>ϒ</sub>.x<sub>TP</sub> )**

$$Cv\  = \ \frac{\mathbf{W}}{\mathbf{N}_{\mathbf{6}}\mathbf{F}_{\mathbf{P}}\mathbf{Y}\sqrt{\mathbf{x}\mathbf{P}_{\mathbf{1}}\mathbf{\rho}_{\mathbf{1}}}}$$

$$Cv\  = \ \frac{\mathbf{W}}{\mathbf{N}_{\mathbf{8}}{\mathbf{F}_{\mathbf{P}}\mathbf{P}}_{\mathbf{1}}\mathbf{Y}}\sqrt{\frac{\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{xM}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{9}}{\mathbf{F}_{\mathbf{P}}\mathbf{P}}_{\mathbf{1}}\mathbf{Y}}\sqrt{\frac{\mathbf{M}\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{x}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{7}}{\mathbf{F}_{\mathbf{P}}\mathbf{P}}_{\mathbf{1}}\mathbf{Y}}\sqrt{\frac{\mathbf{Gg}\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{x}}}$$

**<u>Choked flow (without fittings)</u> (ΔP/P<sub>1</sub>\>
F<sub>ϒ</sub>.x<sub>TP</sub> )**

$$Cv\  = \ \frac{\mathbf{W}}{\mathbf{0.667*N}_{\mathbf{6}}\sqrt{\mathbf{F}_{\mathbf{\mathrm{\Upsilon}}}\mathbf{x}_{\mathbf{T}}\mathbf{P}_{\mathbf{1}}\mathbf{\rho}_{\mathbf{1}}}}$$

$$Cv\  = \ \frac{\mathbf{W}}{\mathbf{0.667*}\mathbf{N}_{\mathbf{8}}\mathbf{P}_{\mathbf{1}}}\sqrt{\frac{\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{F}_{\mathbf{\mathrm{\Upsilon}}}\mathbf{x}_{\mathbf{T}}\mathbf{M}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{0.667*}\mathbf{N}_{\mathbf{9}}\mathbf{P}_{\mathbf{1}}}\sqrt{\frac{\mathbf{M}\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{F}_{\mathbf{\mathrm{\Upsilon}}}\mathbf{x}_{\mathbf{T}}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{0.667*N}_{\mathbf{7}}\mathbf{P}_{\mathbf{1}}}\sqrt{\frac{\mathbf{Gg}\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{F}_{\mathbf{\mathrm{\Upsilon}}}\mathbf{x}_{\mathbf{T}}}}$$

**<u>Choked flow (with fittings)</u> (ΔP \>
(F<sub>LP</sub>/F<sub>p</sub>)
<sup>2</sup>(P<sub>1</sub>-F<sub>F</sub>P<sub>v</sub>) )**

$$Cv\  = \ \frac{\mathbf{W}}{{\mathbf{0.667*}\mathbf{F}_{\mathbf{P}}\mathbf{N}}_{\mathbf{6}}\sqrt{\mathbf{F}_{\mathbf{\mathrm{\Upsilon}}}\mathbf{x}_{\mathbf{T}}\mathbf{P}_{\mathbf{1}}\mathbf{\rho}_{\mathbf{1}}}}$$

$$Cv\  = \ \frac{\mathbf{W}}{\mathbf{0.667*}\mathbf{F}_{\mathbf{P}}\mathbf{N}_{\mathbf{8}}\mathbf{P}_{\mathbf{1}}}\sqrt{\frac{\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{F}_{\mathbf{\mathrm{\Upsilon}}}\mathbf{x}_{\mathbf{T}}\mathbf{M}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{0.667*}\mathbf{F}_{\mathbf{P}}\mathbf{N}_{\mathbf{9}}\mathbf{P}_{\mathbf{1}}}\sqrt{\frac{\mathbf{M}\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{F}_{\mathbf{\mathrm{\Upsilon}}}\mathbf{x}_{\mathbf{T}}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{{\mathbf{0.667*}\mathbf{F}_{\mathbf{P}}\mathbf{N}}_{\mathbf{7}}\mathbf{P}_{\mathbf{1}}}\sqrt{\frac{\mathbf{Gg}\mathbf{T}_{\mathbf{1}}\mathbf{Z}}{\mathbf{F}_{\mathbf{\mathrm{\Upsilon}}}\mathbf{x}_{\mathbf{T}}}}$$

**<u>Non-Turbulent flow (without fittings)</u>**

$$Cv\  = \ \frac{\mathbf{W}}{\mathbf{N}_{\mathbf{27}}\mathbf{F}_{\mathbf{R}}}\sqrt{\frac{\mathbf{T}_{\mathbf{1}}}{\mathbf{\Delta P}\left( \mathbf{P}_{\mathbf{1}}\mathbf{+}\mathbf{P}_{\mathbf{2}} \right)\mathbf{M}}}$$

$$Cv\  = \ \frac{\mathbf{Q}}{\mathbf{N}_{\mathbf{22}}\mathbf{F}_{\mathbf{R}}}\sqrt{\frac{\mathbf{M}\mathbf{T}_{\mathbf{1}}}{\mathbf{\Delta P}\left( \mathbf{P}_{\mathbf{1}}\mathbf{+}\mathbf{P}_{\mathbf{2}} \right)}}$$

W in kg/hr

Q in m<sup>3</sup>/hr for liquids 

P in bar

ρ in kg/m<sup>3</sup>



N<sub>6</sub> : 27.3

N<sub>7</sub> : 417

N<sub>8</sub> : 94.8

N<sub>9</sub> : 2120

N<sub>27</sub>: 0.67

N<sub>22</sub>: 1500

**Link: [More on the Constants mentioned and developped webapp](https://www.linkedin.com/posts/ahmed-hassan-aprco_calculationsguidlines-for-control-valve-activity-7153106921200234496-Kya8/?utm_source=share&utm_medium=member_desktop)**

In [9]:
Q = 38/36 # Volumetric flow rate of the gas at 273.15 K and 1 atm specifically [Nm^3/s]
m_wt = 44.01 # Molecular weight of the gas [g/mol]
viscosity = 1.4665E-4 # Viscosity of the fluid at inlet conditions [Pa*s]
compressibility_factor = 0.988 # Compressibility factor at inlet conditions, [-]
k = 1.3 # Specific heat capacity ratio [-]
T_inlet = 159.85+273.15  # Temperature of the gas at the inlet [K]
P1 = 680E3 # Inlet pressure of the gas before valves and reducers [Pa]
P2 = 310E3 # Outlet pressure of the gas after valves and reducers [Pa]
D1 = 0.08 # Diameter of the pipe before the valve [m]
D2 = 0.1 # Diameter of the pipe after the valve [m]
d = 0.05 # Diameter of the valve [m]

Calculated_Cv_gas =fluids.size_control_valve_g(T=T_inlet, MW=m_wt, mu=viscosity, gamma= k,
                                                Z=compressibility_factor, P1=P1, P2=P2, Q=Q, D1=D1, D2=D2, d=d,
                                                FL=0.85, Fd=0.42, xT=0.60)
print('Calculated Cv is {:.2f}'.format(Calculated_Cv_gas))

Calculated Cv is 72.59


**Safety Relief Sizing** <br>
**Link: [More on the Calculations mentioned and developped webapp](https://www.linkedin.com/feed/update/urn:li:activity:7207477603044175872/)**
1. ***<u>Gases</u>*** <br>

Example 1 from API Standard 520, Part 1 - Sizing and Selection for critical flow
***<u>Gases</u>***

$$\frac{P_{Cf}}{P_{1}} = \left\lbrack \frac{2}{k + 1} \right\rbrack^{\frac{k}{k - 1}}$$

***<u>Subcritical Flow</u>***

$$A = \ \frac{17.9*W*\sqrt{TZ}}{F_{2}K_{d}K_{c}\sqrt{MP_{1}(P_{1} - P_{2})}}$$

$$A = \ \frac{1.998*Q*\sqrt{TMZ}}{F_{2}K_{d}K_{c}\sqrt{P_{1}(P_{1} - P_{2})}}$$

$$A = \ \frac{10.75*Q*\sqrt{TGZ}}{F_{2}K_{d}K_{c}\sqrt{P_{1}(P_{1} - P_{2})}}$$

***<u>Critical Flow</u>***

$$A = \ \frac{13.160*W*\sqrt{TZ}}{CK_{d}P_{1}K_{b}K_{c}\sqrt{M}}$$

$$A = \ \frac{1.46875*Q*\sqrt{TMZ}}{CK_{d}P_{1}K_{b}K_{c}}$$

$$A = \ \frac{7.90625*Q*\sqrt{TGZ}}{CK_{d}P_{1}K_{b}K_{c}}$$

In [10]:
from fluids.safety_valve import API526_letters,API526_A

In [11]:
mass_flow = 24270/3600. # relieved load at kg/s
relief_temperature = 74.85 + 273.15 # K
compressibility_factor = 0.9 
m_wt = 51
k = 1.11 # Isentropic coefficient or ideal gas heat capacity ratio [-]
relief_pressure = 670E3 # Upstream relieving pressure; the set pressure plus the allowable overpressure, plus atmospheric pressure, [Pa]
back_pressure = 101.23E3

critical_P = fluids.compressible.P_critical_flow(relief_pressure,k)
is_critical =fluids.compressible.is_critical_flow(relief_pressure, back_pressure, k)
print('Calculated P Critical is {:0.2f}kPa\nWhile Back Pressure is {:0.2f}kPa'.format(critical_P/1000,back_pressure/1000))
if is_critical:
    print('Critical Flow')
    C = fluids.safety_valve.API520_C(k)
    print('C = {:0.2f}'.format(C))
else:
    print('Sub-Critical Flow')
    F2 = fluids.safety_valve.API520_F2(k, relief_pressure, back_pressure)
    print('F2 = {:.2f}'.format(F2))

Safety_valve_orifice = fluids.safety_valve.API520_A_g(m=mass_flow, T=relief_temperature, Z=compressibility_factor, MW=m_wt, k=k, P1=relief_pressure,P2=back_pressure, Kd=0.975,Kb=1, Kc=1)*1E4 # Cm2
selected_orifice_size = fluids.safety_valve.API520_round_size(Safety_valve_orifice/1E4)*1E4 
Orifice_Letter = API526_letters[API526_A.index(selected_orifice_size/1E4)]

print('Calculated Orifice Area is {:0.2f} cm²\nSelected Orifice Area is {:.2f} cm²'.format(Safety_valve_orifice,selected_orifice_size))
print('PSV Letter: {}'.format(Orifice_Letter))

Calculated P Critical is 390.33kPa
While Back Pressure is 101.23kPa
Critical Flow
C = 0.02
Calculated Orifice Area is 36.99 cm²
Selected Orifice Area is 41.16 cm²
PSV Letter: P


2. ***<u>Steam</u>*** <br>

Example 4 from the 10th edition of API Standard 520, Part 1 - Sizing and Selection:

$$A = \ \frac{190.4*W}{P_{1}K_{d}{K_{sh}K}_{N}K_{b}K_{c}}$$

W in kg/hr, A in mm<sup>2</sup>, Pressures in kPa, T in kelvin, Q in
m<sup>3</sup>/hr for liquids and Nm<sup>3</sup>/hr for gases

G is specific gravity and k is heat capacity ratio.




In [12]:
mass_flow = 69615/3600 # relieved load at kg/s
relief_temperature = 433.8889 + 273.15 # K
relief_pressure = 12236E3 # Upstream relieving pressure; the set pressure plus the allowable overpressure, plus atmospheric pressure, [Pa]


Safety_valve_orifice = fluids.safety_valve.API520_A_steam(m=mass_flow, T=relief_temperature, P1=relief_pressure, Kd=0.975, Kb=1, Kc=1, edition='10E')*1E4 # Cm2
selected_orifice_size = fluids.safety_valve.API520_round_size(Safety_valve_orifice/1E4)*1E4 
Orifice_Letter = API526_letters[API526_A.index(selected_orifice_size/1E4)]

print('Calculated Orifice Area is {:0.2f} cm²\nSelected Orifice Area is {:.2f} cm²'.format(Safety_valve_orifice,selected_orifice_size))
print('PSV Letter: {}'.format(Orifice_Letter))

Calculated Orifice Area is 12.85 cm²
Selected Orifice Area is 18.41 cm²
PSV Letter: L


3. ***<u>Liquids</u>*** <br>

Example 5 in API Standard 520, Part 1 - Sizing and Selection., 10th edition. The calculation involves numerous steps, shown below and ending with a recalculation with a viscosity correction. <br>


$$A = \ \frac{196.33Q\sqrt{G}}{K_{d}K_{w}\ K_{v}K_{c}K_{p}\sqrt{P_{1} - P_{2}}}$$
W in kg/hr, A in mm<sup>2</sup>, Pressures in kPa, T in kelvin, Q in
m<sup>3</sup>/hr for liquids and Nm<sup>3</sup>/hr for gases

G is specific gravity and k is heat capacity ratio.

In [13]:
Q = 6814*1.6666666666666667e-05 # L/min to m^3/s
rho = 0.9*999 # specific gravity times density of water kg/m^3
m = rho*Q # mass flow rate, kg/s
overpressure = 0.1 # Assumed 10% Overpressure
P_design_g = 1724E3 # design pressure, Pa.guage
P1 = (1+overpressure)*P_design_g + 101325.0 # upstream relieving pressure, Pa
backpressure = 0.2
mu = 0.388 # viscosity, Pa*s, converted from 2000 Saybolt Universal Seconds
P2 = backpressure*P_design_g + 101325.0 # backpressure, Pa

Safety_valve_orifice =  fluids.safety_valve.API520_A_l(m=m, rho=rho, P1=P1, P2=P2, overpressure=overpressure, Kd=0.65, Kw=0.97, Kc=1.0, Kv=1.0)*1E4 # Cm2
selected_orifice_size = fluids.safety_valve.API520_round_size(Safety_valve_orifice/1E4)*1E4 
Orifice_Letter = API526_letters[API526_A.index(selected_orifice_size/1E4)]

print('Calculated Orifice Area is {:0.2f} cm²\nSelected Orifice Area is {:.2f} cm²'.format(Safety_valve_orifice,selected_orifice_size))
print('PSV Letter: {}'.format(Orifice_Letter))

Calculated Orifice Area is 30.66 cm²
Selected Orifice Area is 41.16 cm²
PSV Letter: P


**Orifice Sizing**

In [14]:
mass_flow = 27720/3600   # kg/s
D_upstream = 0.07366     # Upstream internal pipe diameter, [m]
D_Orifice = 0.05         #Diameter of orifice, or venturi meter orifice, or flow tube orifice, or cone meter end diameter, or wedge meter fluid flow height, [m]
P_upstream  = 200000.0   # Static pressure of fluid upstream of differential pressure meter at the cross-section of the pressure tap, [Pa]
P_downstream = 183000.0  # Static pressure of fluid downstream of differential pressure meter or at the prescribed location (varies by type of meter) [Pa]

Density =999.1 # kg/m3
viscosity = 0.0011  # Pa.s
k = 1.33 # Isentropic exponent of fluid; required unless epsilon_specified is specified , [-]

Case = 'Estimate Orifice Diameter'
if Case == 'Estimate Mass Flow Rate':
    m = None
    unit = 'kg/s'
elif Case == 'Estimate Downstream pressure':
    P_downstream = None 
    unit = 'Pa'
elif Case == 'Estimate Upstream pressure':
    P_upstream = None 
    unit = 'Pa'
elif Case == 'Estimate Orifice Diameter':
    D_Orifice =  None 
    unit = 'm'
result = fluids.flow_meter.differential_pressure_meter_solver(m=mass_flow,D=D_upstream, D2=D_Orifice, P1=P_upstream,
P2=P_downstream, rho=Density, mu=viscosity, k=k,
meter_type='ISO 5167 orifice', taps='D') # the result is the mass flow rate
print('result for the {} Case is {:0.2f} {}'.format(Case,result,unit))

result for the Estimate Orifice Diameter Case is 0.05 m



**<u>Heat Exchangers Rating</u>**

$Q = F_{t}UA{\mathrm{\Delta}T}_{LMTD} = mC_{p}\mathrm{\Delta}T$ Assuming
no phase change

$${\mathrm{\Delta}T}_{LMTD} = \ \frac{{\mathrm{\Delta}T}_{1} - {\mathrm{\Delta}T}_{2}}{\ln\frac{{\mathrm{\Delta}T}_{1}}{{\mathrm{\Delta}T}_{2}}}$$

$${\mathrm{\Delta}T}_{1} = T_{1} - t_{2}$$

$${\mathrm{\Delta}T}_{2} = T_{2} - t_{1}$$

$$R = \frac{T_{1} - T_{2}}{t_{1} - t_{2}}$$

$$S = \frac{t_{2} - t_{1}}{T_{1} - t_{2}}\ $$

$$F_{t} = \frac{\sqrt{R^{2} + 1}*\ln{(\frac{1 - S}{1 - R})}}{(R - 1)*\ln{(\frac{2 - S(R + 1 - \sqrt{R^{2} + 1})}{2 - S(R + 1 + \sqrt{R^{2} + 1})})}}\ $$

Given Flow rates, Temperatures in and out (T for Hot and t for Cold) and
the exchanger Area A

We Could Estimate U and compare it to the design values

$$Over\ Design\ OD\% = \ \frac{U_{Actual} - U_{required}}{U_{required}}*100\%$$

T in <sup>o</sup>C, U in kj/hr.m<sup>2</sup>.<sup>o</sup>C, kj/kg.
<sup>o</sup>C, A in m<sup>2</sup>

**Link: [More on Shell and tube heat exchangers' Sizing and Rating and developped webapp](https://www.linkedin.com/posts/ahmed-hassan-aprco_shell-and-tube-sizing-and-rating-calculations-activity-7106368828766363648-pa2m/?utm_source=share&utm_medium=member_desktop)**

**More on Shell and tube heat exchangers' Sizing and Rating: [Energy and Process Optimization for the Process Industries, Chapter 6:Heat Exchanger Performance Assessment](https://www.wiley.com/en-us/Energy+and+Process+Optimization+for+the+Process+Industries-p-9781118782507)**

In [30]:
Hot_inlet_T,Hot_outlet_T = 45,35
Cold_inlet_T,Cold_outlet_T = 15,35.03
Cold_flow_rate = 25000 #kg/hr
Hot_flow_rate = 50000 #kg/hr
number_of_shells = 1 # Number of shell-side passes
Cold_Cp = 4.179 #kJ/kg.C
Hot_Cp = 4.183
A = 60.32
U_actual = 3100 #kJ/C.h.m2 actual value from U Calculations

minimum_approach = min(Hot_inlet_T-Cold_outlet_T,Hot_outlet_T-Cold_inlet_T)

dTlm = ht.LMTD(Tci=Cold_inlet_T, Tco=Cold_outlet_T, Thi=Hot_inlet_T, Tho=Hot_outlet_T)

Ft = ht.F_LMTD_Fakheri(Tci=Cold_inlet_T, Tco=Cold_outlet_T, Thi=Hot_inlet_T, Tho=Hot_outlet_T, shells=number_of_shells)

Q = Hot_flow_rate*Hot_Cp*(Hot_inlet_T-Hot_outlet_T) #kJ/hr

UA = Q/(dTlm*Ft) #kJ/C.h

U_required = UA/A 

Over_Design = (U_actual-U_required)*100/U_required

print('Minimum Approach is {:0.2f} C'.format(minimum_approach))
print('logarithmic mean temperature difference (Uncorrected) {:0.2f}'.format(dTlm))
print('Correction Factor Ft {:0.2f}'.format(Ft))
print('logarithmic mean temperature difference (Corrected) {:0.2f}'.format(dTlm*Ft))
print('Calculated Duty {:0.2f} kJ/hr'.format(Q))
print('Calaulated UA {:0.2f} kJ/C.h'.format(UA))
print('Calaulated U {:0.2f} kJ/C.h.m2'.format(U_required))
print('Calaulated Over Design is {:0.2f} %'.format(Over_Design))



Minimum Approach is 9.97 C
logarithmic mean temperature difference (Uncorrected) 14.41
Correction Factor Ft 0.80
logarithmic mean temperature difference (Corrected) 11.59
Calculated Duty 2091500.00 kJ/hr
Calaulated UA 180516.26 kJ/C.h
Calaulated U 2992.64 kJ/C.h.m2
Calaulated Over Design is 3.59 %


**The NTU method**

The NTU Method is very useful when the log mean temperature difference (LMTD) is not available. <br>
Could be used to estimate the expected outlet temperatures of the Heat Exchanger <br>
Or, Given all Temperatures, inlets and outlets, Could be used to estimate the heat exchanger Effectivenss. <br>
The effectiveness is the ratio between the heat transfered and maximum possible heat transfer 

In [31]:
NTU_Method = ht.effectiveness_NTU_method(mh=Hot_flow_rate/3600, mc=Cold_flow_rate/3600, Cph=Hot_Cp*1000, Cpc=Cold_Cp*1000,subtype='S&T', Tci=Cold_inlet_T, Thi=Hot_inlet_T,UA=UA* 0.2777777778)
for i in range(5):
    column = list(NTU_Method.keys())[i]
    NTU_Method[column] = NTU_Method[column] * 0.2777777778
actual_df = pd.DataFrame(NTU_Method,index=['Desired'])
NTU_Method = ht.effectiveness_NTU_method(mh=Hot_flow_rate/3600, mc=Cold_flow_rate/3600, Cph=Hot_Cp*1000, Cpc=Cold_Cp*1000,subtype='S&T', Tci=Cold_inlet_T, Thi=Hot_inlet_T,UA=U_actual*A* 0.2777777778)
for i in range(5):
    column = list(NTU_Method.keys())[i]
    NTU_Method[column] = NTU_Method[column] * 0.2777777778
service_df = pd.DataFrame(NTU_Method,index=['Design/Service'])
pd.concat([actual_df,service_df])

,Q,UA,Cr,Cmin,Cmax,effectiveness,NTU,Thi,Tho,Tci,Tco
Desired,161425.363934,13928.723841,0.138756,8061.342593,16138.117285,0.667488,1.727842,45,34.997262,15,35.024625
Design/Service,163021.449187,14428.395064,0.138756,8061.342593,16138.117285,0.674087,1.789825,45,34.898360,15,35.222618
